In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Path to your zip file inside Google Drive
zip_path = "/content/drive/MyDrive/Task5/food_dataset.zip"

print("Zip File Exists:", os.path.exists(zip_path))


Mounted at /content/drive
Zip File Exists: True


In [4]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/Task5/food_dataset.zip"
extract_path = "/content/food11"


os.makedirs(extract_path, exist_ok=True)


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction Completed!")
print("Folders inside extracted path:", os.listdir(extract_path))


Extraction Completed!
Folders inside extracted path: ['training', 'evaluation', 'validation']


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


base_path = "/content/food11"

train_dir = base_path + "/training"
val_dir = base_path + "/validation"
test_dir = base_path + "/evaluation"


IMG_SIZE = (224, 224)
BATCH_SIZE = 32


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    horizontal_flip=True
)


test_val_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = test_val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_val_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

train_generator.class_indices


Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
Found 3347 images belonging to 11 classes.


{'Bread': 0,
 'Dairy product': 1,
 'Dessert': 2,
 'Egg': 3,
 'Fried food': 4,
 'Meat': 5,
 'Noodles-Pasta': 6,
 'Rice': 7,
 'Seafood': 8,
 'Soup': 9,
 'Vegetable-Fruit': 10}

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

num_classes = train_generator.num_classes

model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 11)             │         2,827 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,787,339 (98.37 MB)

 Trainable params: 25,786,891 (98.37 MB)

 Non-trainable params: 448 (1.75 KB)

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    "best_food_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max"
)

# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stop, checkpoint]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step - accuracy: 0.1525 - loss: 10.7197

309/309 ━━━━━━━━━━━━━━━━━━━━ 167s 508ms/step - accuracy: 0.1525 - loss: 10.7005 - val_accuracy: 0.1181 - val_loss: 9.1237
Epoch 2/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step - accuracy: 0.1491 - loss: 2.3483

309/309 ━━━━━━━━━━━━━━━━━━━━ 147s 475ms/step - accuracy: 0.1491 - loss: 2.3483 - val_accuracy: 0.1534 - val_loss: 2.3166
Epoch 3/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 210s 500ms/step - accuracy: 0.1421 - loss: 2.3285 - val_accuracy: 0.1464 - val_loss: 2.3120
Epoch 4/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 147s 475ms/step - accuracy: 0.1530 - loss: 2.3007 - val_accuracy: 0.1458 - val_loss: 2.3020
Epoch 5/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 148s 478ms/step - accuracy: 0.1536 - loss: 2.2946 - val_accuracy: 0.1461 - val_loss: 2.3002
Epoch 6/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 146s 472ms/step - accuracy: 0.1445 - loss: 2.2947 - val_accuracy: 0.1461 - val_loss: 2.2991
Epoch 7/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 147s 476ms/step - accuracy: 0.1510 - loss: 2.2942 - val_accuracy: 0.1458 - val_loss: 2.2979
Epoch 8/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 147s 475ms/step - accuracy: 0.1547 - loss: 2.2905 - val_accuracy: 0.1455 - val_loss: 2.2976
Epoch 9/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 158s 512ms/step - accuracy: 0.1507 - loss: 2.28

In [8]:

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")


105/105 ━━━━━━━━━━━━━━━━━━━━ 13s 126ms/step - accuracy: 0.2228 - loss: 2.2943
Test Accuracy: 14.97%
Test Loss: 2.2972


In [9]:
from sklearn.metrics import classification_report
import numpy as np

# Get true labels
true_labels = test_generator.classes

# Get predictions
pred_probs = model.predict(test_generator)
pred_labels = np.argmax(pred_probs, axis=1)

# Generate report
print(classification_report(true_labels, pred_labels))


105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 108ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       368
           1       0.00      0.00      0.00       148
           2       0.15      1.00      0.26       500
           3       0.00      0.00      0.00       335
           4       0.00      0.00      0.00       287
           5       0.00      0.00      0.00       432
           6       0.00      0.00      0.00       147
           7       0.00      0.00      0.00        96
           8       0.00      0.00      0.00       303
           9       1.00      0.00      0.00       500
          10       0.00      0.00      0.00       231

    accuracy                           0.15      3347
   macro avg       0.10      0.09      0.02      3347
weighted avg       0.17      0.15      0.04      3347



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:


class_labels = {
    0: "Bread",
    1: "Dairy product",
    2: "Dessert",
    3: "Egg",
    4: "Fried food",
    5: "Meat",
    6: "Noodles-Pasta",
    7: "Rice",
    8: "Seafood",
    9: "Soup",
    10: "Vegetable-Fruit"
}

class_labels



{0: 'Bread',
 1: 'Dairy product',
 2: 'Dessert',
 3: 'Egg',
 4: 'Fried food',
 5: 'Meat',
 6: 'Noodles-Pasta',
 7: 'Rice',
 8: 'Seafood',
 9: 'Soup',
 10: 'Vegetable-Fruit'}

In [17]:
nutrition_data = {
    "Bread":      {"calories": 265, "protein": 9,  "fat": 3.2,  "carbs": 49},
    "Dairy":      {"calories": 103, "protein": 8,  "fat": 2.4,  "carbs": 12},
    "Dessert":    {"calories": 350, "protein": 4,  "fat": 15,   "carbs": 52},
    "Egg":        {"calories": 155, "protein": 13, "fat": 11,   "carbs": 1.1},
    "Fried Food": {"calories": 312, "protein": 7,  "fat": 18,   "carbs": 28},
    "Meat":       {"calories": 250, "protein": 26, "fat": 15,   "carbs": 0},
    "Noodles":    {"calories": 138, "protein": 4.5,"fat": 2.1,  "carbs": 25},
    "Rice":       {"calories": 130, "protein": 2.7,"fat": 0.3,  "carbs": 28},
    "Seafood":    {"calories": 206, "protein": 20, "fat": 12,   "carbs": 0},
    "Soup":       {"calories": 80,  "protein": 4,  "fat": 2,    "carbs": 12},
    "Vegetables": {"calories": 65,  "protein": 3,  "fat": 0.5,  "carbs": 14}
}

nutrition_data


{'Bread': {'calories': 265, 'protein': 9, 'fat': 3.2, 'carbs': 49},
 'Dairy': {'calories': 103, 'protein': 8, 'fat': 2.4, 'carbs': 12},
 'Dessert': {'calories': 350, 'protein': 4, 'fat': 15, 'carbs': 52},
 'Egg': {'calories': 155, 'protein': 13, 'fat': 11, 'carbs': 1.1},
 'Fried Food': {'calories': 312, 'protein': 7, 'fat': 18, 'carbs': 28},
 'Meat': {'calories': 250, 'protein': 26, 'fat': 15, 'carbs': 0},
 'Noodles': {'calories': 138, 'protein': 4.5, 'fat': 2.1, 'carbs': 25},
 'Rice': {'calories': 130, 'protein': 2.7, 'fat': 0.3, 'carbs': 28},
 'Seafood': {'calories': 206, 'protein': 20, 'fat': 12, 'carbs': 0},
 'Soup': {'calories': 80, 'protein': 4, 'fat': 2, 'carbs': 12},
 'Vegetables': {'calories': 65, 'protein': 3, 'fat': 0.5, 'carbs': 14}}

In [18]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predict_food(image_path):
    # Load image
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])

    # Get food name
    food_name = class_labels[predicted_class]

    # Get nutrition info
    nutrients = nutrition_data.get(food_name, "No data available")

    return food_name, nutrients


In [20]:
from google.colab import files

uploaded = files.upload()

for file_name in uploaded.keys():
    image_path = file_name
    food, nutri = predict_food(image_path)
    print("\nPredicted Food:", food)
    print("Nutrition Info:", nutri)


Saving desert.jpg to desert.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

Predicted Food: Dessert
Nutrition Info: {'calories': 350, 'protein': 4, 'fat': 15, 'carbs': 52}


In [21]:
model.save("food_classification_model.h5")


In [22]:
import json

with open("class_labels.json", "w") as f:
    json.dump(class_labels, f)


In [23]:
with open("nutrition_data.json", "w") as f:
    json.dump(nutrition_data, f)


In [24]:
from google.colab import files

files.download("food_classification_model.h5")
files.download("class_labels.json")
files.download("nutrition_data.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>